In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import string
import requests

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [ ]:
response.text[:1500]

'This is the 100th Etext file presented by Project Gutenberg, and\nis presented in cooperation with World Library, Inc., from their\nLibrary of the Future and Shakespeare CDROMS.  Project Gutenberg\noften releases Etexts that are NOT placed in the Public Domain!!\n\nShakespeare\n\n*This Etext has certain copyright implications you should read!*\n\n<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM\nSHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\nPROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE\nWITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE\nDISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS\nPERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED\nCOMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY\nSERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>\n\n*Project Gutenberg is proud to cooperate with The World Library*\nin the presentation of The Complete Works of William Shakespeare\nfor your

In [ ]:
data = response.text.split('\n')
data[0]

'This is the 100th Etext file presented by Project Gutenberg, and'

In [ ]:
data = data[253:]
data[0]

'  From fairest creatures we desire increase,'

In [ ]:
len(data)

124204

In [ ]:
data = " ".join(data)
data[:1000]

"  From fairest creatures we desire increase,   That thereby beauty's rose might never die,   But as the riper should by time decease,   His tender heir might bear his memory:   But thou contracted to thine own bright eyes,   Feed'st thy light's flame with self-substantial fuel,   Making a famine where abundance lies,   Thy self thy foe, to thy sweet self too cruel:   Thou that art now the world's fresh ornament,   And only herald to the gaudy spring,   Within thine own bud buriest thy content,   And tender churl mak'st waste in niggarding:     Pity the world, or else this glutton be,     To eat the world's due, by the grave and thee.                        2   When forty winters shall besiege thy brow,   And dig deep trenches in thy beauty's field,   Thy youth's proud livery so gazed on now,   Will be a tattered weed of small worth held:     Then being asked, where all thy beauty lies,   Where all the treasure of thy lusty days;   To say within thine own deep sunken eyes,   Were an al

In [ ]:
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

tokens = clean_text(data)
print(tokens[:50])

['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die', 'but', 'as', 'the', 'riper', 'should', 'by', 'time', 'decease', 'his', 'tender', 'heir', 'might', 'bear', 'his', 'memory', 'but', 'thou', 'contracted', 'to', 'thine', 'own', 'bright', 'eyes', 'feedst', 'thy', 'lights', 'flame', 'with', 'selfsubstantial', 'fuel', 'making', 'a', 'famine', 'where', 'abundance', 'lies', 'thy']


In [ ]:
len(tokens)

898199

In [ ]:
len(set(tokens))

27956

In [ ]:
length = 50 + 1
lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 20000: # dectrased the sized from 2000000
    break

print(len(lines))

19951


In [ ]:
lines[0]

'from fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self'

In [ ]:
tokens[50]

'self'

In [ ]:
lines[1]

'fairest creatures we desire increase that thereby beautys rose might never die but as the riper should by time decease his tender heir might bear his memory but thou contracted to thine own bright eyes feedst thy lights flame with selfsubstantial fuel making a famine where abundance lies thy self thy'

**Build LSTM Model and Prepare X and y**

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [ ]:
sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:,-1]
X[0]

array([  37,  561, 1527,  130,  236,  461,    8, 1526,  164,  459,  104,
        154,  206,   16,   25,    2, 1525,   69,   31,   51,  959,   24,
        385,  958,  104,  235,   24,  333,   16,   10, 1524,    3,   68,
         88,  218,   54, 3523,    9, 3522,  957,   13, 3521, 3520,  234,
         11, 3518,   73,  559,  233,    9])

In [ ]:
y

array([  41,    9, 1528, ...,   72,   11,  719])

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
y = to_categorical(y, num_classes=vocab_size)

In [ ]:
seq_length = X.shape[1]
seq_length


50

## LSTM Model

**https://kgptalkie.com/text-generation-using-tensorflow-keras-and-lstm/**

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            176400    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 3528)              356328    
                                                                 
Total params: 683628 (2.61 MB)
Trainable params: 683628 (2.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(X, y, batch_size = 256, epochs = 10)

Epoch 1/10
78/78 [==============================] - 41s 463ms/step - loss: 7.0268 - accuracy: 0.0212
Epoch 2/10
78/78 [==============================] - 35s 445ms/step - loss: 6.5435 - accuracy: 0.0274
Epoch 3/10
78/78 [==============================] - 35s 443ms/step - loss: 6.5182 - accuracy: 0.0266
Epoch 4/10
78/78 [==============================] - 36s 457ms/step - loss: 6.4501 - accuracy: 0.0273
Epoch 5/10
78/78 [==============================] - 36s 456ms/step - loss: 6.3845 - accuracy: 0.0270
Epoch 6/10
78/78 [==============================] - 34s 441ms/step - loss: 6.3434 - accuracy: 0.0299
Epoch 7/10
78/78 [==============================] - 36s 460ms/step - loss: 6.2708 - accuracy: 0.0349
Epoch 8/10
78/78 [==============================] - 36s 460ms/step - loss: 6.1894 - accuracy: 0.0351
Epoch 9/10
78/78 [==============================] - 35s 447ms/step - loss: 6.1198 - accuracy: 0.0382
Epoch 10/10
78/78 [==============================] - 35s 451ms/step - loss: 6.0578 - accura

In [ ]:
seed_text=lines[1343]
seed_text

'longer yours than you your self here live against this coming end you should prepare and your sweet semblance to some other give so should that beauty which you hold in lease find no determination then you were your self again after your selfs decease when your sweet issue your sweet'

In [ ]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
  text = []

  for _ in range(n_words):
    encoded = tokenizer.texts_to_sequences([seed_text])[0]
    encoded = pad_sequences([encoded], maxlen = text_seq_length, truncating='pre')

    y_predict = model.predict_classes(encoded)

    predicted_word = ''
    for word, index in tokenizer.word_index.items():
      if index == y_predict:
        predicted_word = word
        break
    seed_text = seed_text + ' ' + predicted_word
    text.append(predicted_word)
  return ' '.join(text)

In [ ]:
def generate_text_seq(model, tokenizer, text_seq_length, seed_text, n_words):
    text = []

    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([seed_text])[0]
        encoded = pad_sequences([encoded], maxlen=text_seq_length, truncating='pre')

        y_predict = np.argmax(model.predict(encoded), axis=-1)

        predicted_word = ''
        for word, index in tokenizer.word_index.items():
            if index == y_predict:
                predicted_word = word
                break
        seed_text = seed_text + ' ' + predicted_word
        text.append(predicted_word)
    return ' '.join(text)

In [ ]:
generate_text_seq(model, tokenizer, seq_length, seed_text, 100)

1/1 [==============================] - 0s 27ms/step


'love and my love and my love and my love'